In [2]:
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt
import warnings
from IPython.display import display
from ipywidgets import interactive
import ipywidgets as widgets

In [27]:
class BoostConverter:
    def __init__(self, V_i, L, R_L, C, R, D, T):
        self.V_i = V_i
        self.L = L
        self.R_L = R_L
        self.C = C
        self.R = R
        self.D = D
        self.T = T
                
    def _switched_dynamics(self, t, y):
        def switch_closed(t, y):
            A = np.array([[- 1 / (self.R * self.C), 0],
                         [0, - self.R_L / self.L]])
            B = np.array([0, 1 / self.L])
            return A @ y + B * self.V_i
        
        def switch_open(t, y):
            A = np.array([[-1 / (self.R * self.C), 1 / self.C],
                         [-1 / self.L, - self.R_L / self.L]])
            B = np.array([0, 1 / self.L])
            return A @ y + B * self.V_i
        
        if (t - (t // self.T) * self.T) > self.D * self.T:
            return switch_open(t, y)
        else:
            return switch_closed(t, y)
        
    def _averaged_dynamics(self, t, y):
        A_closed = np.array([[- 1 / (self.R * self.C), 0],
                            [0, - self.R_L / self.L]])
        B_closed = np.array([0, 1 / self.L])
        
        A_open = np.array([[-1 / (self.R * self.C), 1 / self.C],
                         [-1 / self.L, - self.R_L / self.L]])
        B_open = np.array([0, 1 / self.L])
        
        A_avg = self.D * A_closed + (1 - self.D) * A_open
        B_avg = self.D * B_closed + (1 - self.D) * B_open
        
        return A_avg @ y + B_avg * self.V_i
    
    @property
    def w_n(self):
        return np.sqrt(self.R_L / (self.R * self.L * self.C) + ((1 - self.D) ** 2) / (self.L * self.C))

    @property
    def damping_ratio(self):
        return 0.5 * (self.R_L / self.L + 1 / (self.R * self.C)) / self.w_n
    
    @property
    def w_d(self):
        if self.damping_ratio >= 1:
            warnings.warn("System is not under-damped, so w_d is meaningless")
        return self.w_n * np.sqrt(1 - self.damping_ratio ** 2)
    
    @property
    def time_to_peak(self):
        return np.pi / self.w_d
    
    @property
    def pct_overshoot(self):
        return np.exp(- np.pi * self.w_n * self.damping_ratio / self.w_d)
    
    @property
    def settling_time(self):
        return -1 / (self.damping_ratio * self.w_n) * np.log(0.02 * np.sqrt(1 - self.damping_ratio ** 2))
        
    def simulate_averaged_dynamics(self, time_len):
        sol = integrate.solve_ivp(self._averaged_dynamics, (0, time_len), [0, 0], max_step=0.01 * time_len)
        if not sol.success:
            print(f"Failed to integrate averaged dynamics: {sol.message}")
            return

        return sol.t, sol.y
    
    def simulate_switching_dynamics(self, time_len):
        sol = integrate.solve_ivp(self._switched_dynamics, (0, time_len), [0, 0], max_step=0.1 * self.T)
        if not sol.success:
            print(f"Failed to integrate switching dynamics: {sol.message}")
            return

        return sol.t, sol.y
    
    def solve_ripples(self):
        """
        V_i D = V_o(DT) (1-D) + R_L (1-D) I_L(DT) + R_L D I_L(0)
        -D = (1-D) V_o(DT) - R (1-D) I_L(DT)
        (R_L / L * DT - 1)I_L(0) + I_L(DT) = V_i/L * DT
        DT / RC = V_o(0) - V_o(DT)
        """
        
        A = np.array([[0, 1 - self.D, self.R_L * self.D, self.R_L * (1 - self.D)],
                      [0, 1 - self.D, 0, -self.R * (1 - self.D)],
                      [0, 0, self.R_L / self.L * self.D * self.T - 1, 1],
                      [1, -1, 0, 0]])
        b = np.array([self.D * self.V_i, 
                      -self.D, 
                      self.V_i / self.L * self.D * self.T,
                      self.D * self.T / (self.R * self.C)
                     ])
        
        return (np.linalg.inv(A) @ b)
    
    @property
    def dcm_gain(self):
        K = 2 * self.L / (self.R * self.T)
        return (1 + np.sqrt(1 + 4 * (self.D ** 2) / K)) / 2

In [4]:
def plot_simulation(fig, V_i, L, R_L, C, R, D, T, t_len):
    fig.clf()
    
    circuit = BoostConverter(V_i, L, R_L, C, R, D, T)
    t, y = circuit.simulate_averaged_dynamics(t_len)
    
    plt.subplot(1, 2, 1)
    plt.plot(t, y[0])
    
    plt.legend(['$V_{o, avg}$'])
    
    plt.axhline(0.98 / (1 - D) * V_i, linestyle='dashed', color='gray')
    plt.axhline(1.02 / (1 - D) * V_i, linestyle='dashed', color='gray')
    
    if circuit.damping_ratio < 1:
        plt.axvline(circuit.time_to_peak, linestyle='dashed', color='black')
        plt.text(circuit.time_to_peak - 0.05,0,'$T_p$')
        
        plt.axvline(circuit.settling_time, linestyle='dashed', color='black')
        plt.text(circuit.settling_time - 0.05,0,'$T_s$')
        
        plt.axhline((1 + circuit.pct_overshoot) / (1 - D) * V_i, linestyle='dashed', color='black')
        plt.text(0, (1 + circuit.pct_overshoot) / (1 - D) * V_i + 0.1,'$V_{o,max}$')
        
    plt.subplot(1, 2, 2)
    plt.plot(t, y[1])
    plt.legend(['$I_{L, avg}$'])
    
    plt.draw()

# High Voltage

$V_o = 200 \ V$, $I_o = 2.5\  mA, V_i = 5 \ V$

In [ ]:
fig = plt.figure(figsize=(15, 7))
plot_simulation(fig, 5, 18e-6, 60e-3, 100e-6, 80e3, 1 - 1 / 40, 1e-4, 5)

# Low voltage

In [39]:
V_i = 3.3
L = 54e-6
R_L = 180e-3
C = 100e-6
R = 10e3
D = 0.5
T = 100e-6

circuit = BoostConverter(V_i, L, R_L, C, R, D, T)

In [40]:
circuit.solve_ripples()

array([ 3.95985359e+00,  3.95980359e+00, -3.66607149e+00,  4.95980359e-04])

In [41]:
circuit.dcm_gain

48.6151204385359